In [1]:
import pandas as pd
import numpy as np
import os
import re
import matplotlib.pyplot as plt
import math
from math import log2
from sklearn.cluster import KMeans
import shutil

##### define different functions of calculating differential score
def ks_score(r1, r2):
    r = r1-r2
    max_ks = max(max(r), 0)
    min_ks = min(min(r), 0)
    return (abs(max_ks)+abs(min_ks))/r1.sum()
def entropy_score(r_true:[],r_false:[]):
    h = 0.0
    for i in range(0,len(r_true)):
        if r_false[i]!=0:
            h = -r_true[i] * log2(r_false[i])+h
        else:
            h = -r_true[i] * log2(1/100000) + h
    return h

def KL(p,q):
    return sum(_p*math.log(_p/_q) for (_p,_q) in zip(p,q) if _p != 0 )
def JS(p,q):
    M = [0.5*(_p +_q) for (_p,_q) in zip(p,q)]
    return 0.5*(KL(p,M)+KL(q,M))



In [2]:
def read_d(dir_name,sim_type):######this function is uesd to collect the results of each bit.
    
    def read_file(name):
        aa=pd.read_csv(name)
        getcol=[columns for columns in aa]
        del aa
        temp=[]
        for c in getcol:
            temp.append(remove(c))
        return dict(temp)

    def remove(text):    ####the result is saved in csv file and there are punctuations in the front and end of the rusult.
        punctuation = '[{} ？!,;+"\'' 
        text = re.sub(r'[{}]+'.format(punctuation), '', text)
        text=re.split(":",text)
        return text
    files=os.listdir(str(dir_name))
    whole_list=[]
    result=[]
    name=[]
    for file in files:
        if  sim_type.search(file):
            name.append(file)
            result.append(read_file(dir_name+"/"+file))
    temp_len=[]
    for i in result:
        temp_len.append(len(i))
    full_dic=result[temp_len.index(max(temp_len))].keys()    ####some of the file lack some keys for no shots on it. Find the full key as a standard example
    for dic in result:
        if len(full_dic - dic.keys()) != 0:
            for key in full_dic - dic.keys():   ###fulfill all lack keys
                dic[key]=0
    for ele in result:
        tem=sorted(list(ele.items() ), key=lambda x:x[0])  #####set the results in ascending order of key 
        pp=[]
        for i in tem:
            pp.append(  float(i[1]) )
        whole_list.append(pp) #####save the name and result together
    return whole_list,name

In [ ]:
#####set the type of backend to collect
noise_file = re.compile("startQiskit_noise_[0-9]+"+".csv") 

In [3]:
####JS score with standardised answers
def formal_js(a,b):
    thershold=0.5
    candidates = [] 
    answer = -1
    for results in a:
        flag = -1
        for i in range(0,len(candidates)):
            if JS(candidates[i]/(candidates[i].sum()),np.asarray(results)/(np.asarray(results).sum()))<thershold:
                flag = i
                candidates[i] = candidates[i] + np.asarray(results)/(np.asarray(results).sum())
                if candidates[i].sum() > candidates[answer].sum():
                    answer = i
                break

        if flag == -1:
            candidates.append(np.asarray(results)/(np.asarray(results).sum()))
            if answer == -1:
                answer = 0

    wrong_out = []           
    print("Right answer:",candidates[answer]/candidates[answer].sum()*1024)

    result=[]

    for i in range(0, len(a)):
        k = JS(np.asarray(a[i])/np.asarray(a[i]).sum(),candidates[answer]/candidates[answer].sum())
        result.append([b[i],k])
        if k > thershold:
            print(k)
            print(b[i]+":"+str(a[i])+":"+str(k))
            wrong_out.append([b[i],k])
            
    return result,wrong_out


js_error=[]
for i in range(1,100):
    print("reading folder"+str(i))
    print("------------------------------------------")
    a,b=read_d(str(i),noise_file)
    js_error.append(["folder"+str(i),formal_js(a,b)])
for i in range(0,len(js_error)):
    if len(js_error[i][1][1])>0:
        print(js_error[i][0])

reading folder1
------------------------------------------
Right answer: [ 47.35204969  81.51453416  49.43503106  58.2757764   76.45813665
  57.76695652  48.60819876  95.53093168  24.58236025  45.65863354
  33.8047205   47.33614907  43.85391304  38.45565217  46.58086957
 113.59403727   9.52447205  10.75677019   7.23478261   7.89465839
  10.48645963   8.74534161   7.87080745  14.17540373   2.4884472
   4.39652174   3.21192547   4.93714286   4.34881988   3.66509317
   4.54757764  10.90782609]
reading folder2
------------------------------------------


C:\Users\ce\AppData\Local\Temp\ipykernel_19728\3527356048.py:14: FutureWarning: Possible nested set at position 1
  text = re.sub(r'[{}]+'.format(punctuation), '', text)


Right answer: [189.62285714  87.30236025  48.13913043  60.19975155  56.47900621
  52.60720497  21.84745342  21.48173913  97.61391304  52.75031056
  45.88124224  64.39751553  39.31428571  52.9331677   15.99602484
  17.72919255  19.07279503   9.07925466   7.06782609   8.66583851
   5.96273292   5.54931677   2.71900621   2.83031056   9.65167702
   5.54931677   4.49987578   6.20124224   3.96720497   5.82757764
   1.41515528   1.64571429]
reading folder3
------------------------------------------
Right answer: [27.9373913  38.1773913  49.87229814 49.0931677  58.75279503 57.1468323
 81.3157764  78.39006211 32.11130435 48.13118012 48.52074534 59.31726708
 49.66559006 61.68645963 90.53018634 93.49565217  3.49813665  4.18186335
  5.52546584  5.4221118   9.07130435  7.68795031  9.22236025  8.41937888
  3.3868323   4.79403727  4.39652174  6.07403727  4.61118012  6.05018634
  8.97590062  8.53863354]
reading folder4
------------------------------------------
Right answer: [284.57341615 113.22832298

Right answer: [ 47.22484472  32.65987578  61.01863354  43.55975155 104.15701863
  53.42608696  62.95055901  79.35204969  31.6978882   24.9242236
  45.14186335  45.05440994  57.94981366  41.00770186  70.81341615
 116.27329193   8.42732919   5.70037267   7.19503106   6.51130435
  11.25763975   6.13763975   8.14906832  10.49440994   2.90186335
   2.40099379   4.61913043   4.52372671   5.69242236   4.00695652
   7.24273292  11.52795031]
reading folder21
------------------------------------------
Right answer: [ 31.97614907  50.64347826  38.6226087   51.4226087   25.68745342
  33.97962733  26.00546584  37.35850932  92.48596273  82.57192547
  58.2757764   77.57913043  81.88819876  53.26708075  51.34310559
 131.82409938   3.64124224   5.77987578   5.7242236    6.47950311
   2.7826087    3.92745342   4.22956522   6.71801242   9.02360248
   7.69590062   5.45391304   7.10757764   8.16496894   4.90534161
   4.82583851  12.60919255]
reading folder22
------------------------------------------
Right

Right answer: [ 10.0173913   18.90583851  21.0047205   49.84049689  15.85291925
  30.42583851  33.30385093  82.69913043  19.14434783  47.5110559
  38.87701863 108.28322981  36.03875776 107.78236025  79.80521739
 183.75552795   2.70310559   3.97515528   7.33813665  16.1152795
   4.20571429   6.01838509  12.9510559   26.46658385   1.72521739
   4.65888199   3.77639752  10.74086957   3.49018634  10.70906832
   7.95826087  17.92      ]
reading folder39
------------------------------------------
Right answer: [ 84.80596273 107.65515528 102.36819876  61.6626087   99.07677019
  45.76993789  39.11552795  25.13093168  94.7915528   66.21813665
  52.44024845  34.64745342  55.74757764  31.83304348  23.80322981
   9.61192547   8.89639752  10.32745342   9.5642236    5.89913043
   9.21440994   4.19776398   3.77639752   2.24993789   8.93614907
   6.54310559   5.00869565   3.49018634   5.05639752   2.85416149
   2.31354037   0.99378882]
reading folder40
------------------------------------------
Right 

Right answer: [ 13.86534161  29.65465839  31.31627329  50.79453416  28.41440994
  55.4931677   69.40621118 109.8573913   24.04968944  45.34857143
  49.10906832  73.40521739  47.69391304  65.6536646   80.08347826
 124.20770186   2.72695652   8.37167702   5.21540373  11.86981366
   5.23130435  11.47229814  10.62956522  19.62136646   2.2578882
   4.7463354    4.77813665   7.34608696   4.75428571   6.66236025
   7.86285714  12.10037267]
reading folder56
------------------------------------------
Right answer: [  5.66857143  13.44397516  13.48372671  37.02459627  11.77440994
  24.51875776  26.06111801  86.24496894  12.06857143  30.73590062
  29.20944099  96.43726708  26.68919255  66.67925466  68.5952795
 325.18360248   1.87627329   4.76223602   3.18012422  11.48024845
   3.60149068   8.41142857   7.9821118   33.37540373   1.68546584
   4.92124224   2.92571429   9.42906832   4.22161491  13.83354037
   6.35229814  32.14310559]
reading folder57
------------------------------------------
Right 

Right answer: [ 27.16621118  29.20149068  56.14509317  90.71304348  47.23279503
  52.90136646 105.32571429 169.09515528  34.0352795   23.97813665
  49.35552795  15.90857143  61.40819876  46.67627329  84.19378882
  25.28993789   4.17391304   2.98136646   7.06782609   8.6578882
   9.66757764   5.74012422  16.95006211  16.07552795   3.28347826
   2.43279503   4.88149068   1.48670807   5.78782609   4.97689441
   8.5863354    2.62360248]
reading folder73
------------------------------------------
Right answer: [  4.2136646   10.03329193  11.58360248  37.28695652   7.33018634
  18.46857143  21.58509317  82.03130435  10.92372671  25.87031056
  32.77913043 106.25590062  23.58857143  59.05490683  75.51204969
 374.72993789   1.10509317   2.7110559    2.53614907   8.24447205
   2.1863354    5.21540373   6.16944099  25.36944099   1.10509317
   2.67925466   3.21192547  10.12074534   2.4089441    5.66062112
   7.6084472   36.42037267]
reading folder74
------------------------------------------
Right

Right answer: [  7.10757764  12.00496894  15.94037267  32.77118012  11.98111801
  20.69465839  28.19975155  57.6         21.0842236   39.73565217
  56.31204969 123.27751553  44.88745342  83.58956522 133.02459627
 231.64024845   0.98583851   1.31975155   1.91602484   3.39478261
   2.01142857   2.24993789   3.64124224   5.55726708   3.54583851
   3.72074534   7.08372671  11.79826087   8.67378882   7.96621118
  17.5463354   22.7378882 ]
reading folder90
------------------------------------------
Right answer: [ 25.46484472  48.46509317  48.35378882  61.09018634  39.91055901
  87.54881988  68.2136646   37.27900621  33.00968944  53.26708075
  65.91602484  77.14186335  59.34906832 101.70832298  59.39677019
  34.87006211   5.75602484  10.68521739  12.6489441   16.09937888
   4.53167702  10.17639752   8.41937888   6.01838509   3.14832298
   5.47776398   7.04397516   8.06161491   5.63677019  10.58981366
   5.77192547   2.94956522]
reading folder91
------------------------------------------
Righ

In [4]:
##JS score with trimmed mean 
def js_ternn(a,b):
    thershold=0.6
    candidates = [] 
    for i in range(0,len(a[0])):
        temp=[]
        for c in range(0,len(a)):
            temp.append(a[c][i])        
        fla=sorted(temp)[4:-4]
        candidates.append( sum(fla)/len(fla)  )
    print("Right answer:", np.asarray(candidates)/ np.asarray(candidates).sum()*1024)
    max_diff_name = ''
    result=[]
    wrong_out = [] 
    for i in range(0, len(a)):
        k = JS(np.asarray(a[i])/np.asarray(a[i]).sum(), np.asarray(candidates)/ np.asarray(candidates).sum())
        result.append([b[i],k])
        if k > thershold:
            print(k)
            print(b[i]+":"+str(a[i])+":"+str(k))
            wrong_out.append([b[i],k])
    return result,wrong_out
noise_file = re.compile("startQiskit_noise_[0-9]+"+".csv") 

js_ter=[]
for i in range(1,100):
    print("reading folder"+str(i))
    print("------------------------------------------")
    a,b=read_d(str(i),noise_file)
    js_ter.append(["folder"+str(i),js_ternn(a,b)])
for i in range(0,len(js_ter)):
    if len(js_ter[i][1][1])>0:
        print(js_ter[i][0])

reading folder1
------------------------------------------
Right answer: [ 48.29799992  80.62071203  29.79750023  62.00059704  73.736187
  63.96760419  50.19855413 104.79629317  28.17604839  44.04501149
  23.59079523  52.67060366  41.25398783  41.44005607  51.24851065
 133.9957169    4.69157787   9.34328397   4.30615079   6.83136267
   8.28003686   6.39277324   5.02384259   9.68883928   2.57837424
   4.25298844   2.13978481   5.43585084   3.96059548   3.76123665
   4.90422729  12.57289706]
reading folder2
------------------------------------------
Right answer: [198.05392427  90.51113001  44.09024061  61.04507028  54.78305237
  50.3906758   14.07353309  17.36461408 115.62323045  60.39197639
  44.1286579   63.68305738  39.68505828  47.52218498  12.48561853
  19.36231304  18.56835576   8.27252264   4.49482267   5.58331249
   5.92906808   5.49367215   1.28057626   1.86964134  10.91050973
   6.24921215   4.31554199   6.00590266   3.95698064   5.10949927
   1.01165524   1.75438948]
reading 

Right answer: [ 48.68226476  25.95523144  47.19132989  42.21290388 118.78213309
  54.67193356  68.18109997  88.85971842  27.71842398  19.58958776
  38.80320065  43.0815355   62.73594652  41.13683784  77.54157804
 126.28866606   4.8487795    2.63182417   5.12103717   4.64134508
  11.75893852   5.60072926   6.27489112   8.62149296   2.63182417
   1.90580371   3.6819609    4.48576927   6.00263344   3.83753672
   7.93436645  12.58867619]
reading folder21
------------------------------------------
Right answer: [ 34.70134959  39.43995153  25.54093247  42.50222354  27.86403537
  21.7790897   16.07692803  33.72459042 105.95197154  89.21507109
  62.86897227  86.83917039  87.80273012  58.64514882  53.98574356
 146.88346073   3.44505601   4.97619201   2.62669021   4.07862953
   2.81148249   2.41549904   1.66313049   3.31306152   9.96558347
   8.24965519   5.92655229   7.65568002   8.71163588   5.21378208
   5.21378208  13.91221851]
reading folder22
------------------------------------------
Righ

Right answer: [ 33.09955181  59.67809015  46.3579157  119.64978043  41.53669247
  57.94739463  38.69340697  86.68930237  25.51230628  50.82372825
  49.49480133 100.99844568  40.53227096  47.98044275  68.09977817
  61.31606984   3.21414882   5.98017113   4.80577059  12.05305808
   4.80577059   5.93381321   5.70202363   9.82787813   2.93600133
   4.88303379   4.46581255   9.67335174   4.40400199   4.55852838
   6.55191875   5.79473946]
reading folder41
------------------------------------------
Right answer: [ 30.02176548  29.83080468  60.23449176  49.52704701  63.6172259
  66.41343759 166.65421656 112.926032    19.31432073  14.02197861
  29.89900497  26.66631146  47.24915749  31.99957375  80.44905625
  80.59909688   4.35117819   4.43301853   8.77055666   7.18831004
   9.08427797  10.54376407  21.29212899  15.98614682   1.90960798
   1.66408696   2.93261226   2.7280114    4.74673984   3.53277477
   7.52931147   7.88395295]
reading folder42
------------------------------------------
Right

Right answer: [ 44.40471229  47.83547838  47.83547838  48.51884236  25.81442288
  13.72306435  18.33925775  15.78710249 116.28344569 154.62155941
 185.23347634  52.9397889   61.0564794   37.15268641  44.47444331
  17.92087164   4.65803201   4.6440858    4.78354784   4.75565543
   2.84502554   1.49224379   1.96641471   1.60381342  11.12907048
  15.22925434  17.75351719   4.90906367   6.47103847   3.41681988
   4.40700034   1.99430712]
reading folder61
------------------------------------------
Right answer: [37.93031296 47.9853994  93.85186042 76.53184243 40.81259559 50.24910995
 76.08436476 88.9822505  21.50525031 29.75726496 49.73582675 89.74559476
 28.40167084 44.91886126 73.3205321  81.30932459  3.93517126  5.47502089
 10.87107512  7.30441488  4.38264893  4.84328771  7.56763704  8.55472013
  2.07945505  2.8296382   4.39581004  8.93639226  3.0007326   4.8301266
  6.71216503  7.1596427 ]
reading folder62
------------------------------------------
Right answer: [49.60391619 60.27660776

Right answer: [ 57.63868442  34.44133082  63.91686728  21.92043506  36.14388888
  17.32707529  33.21761721   8.67240513 197.92237482  39.15883545
 166.5669305   40.18746428 109.03465595  32.4372781   54.03848352
  16.22750654   5.90574828   4.3096001    9.38180433   2.66024697
   3.6002009    2.12819758   3.19229637   1.25918357  18.90548849
   3.56473094  15.21661269   4.16772026  11.42132701   2.87306673
   5.05446925   1.50747328]
reading folder83
------------------------------------------
Right answer: [  6.87693889  15.92092773  13.44072026  36.10079757  10.93546019
  25.14028478  25.09017958  71.80075354  11.97514312  32.75627538
  28.00880755 109.20428634  27.31986104  72.74022606  77.08685228
 370.22733278   0.56368352   1.67852425   1.49062974   3.670206
   1.02715663   2.29231296   2.56789157   6.71409698   1.08978813
   3.18168029   2.51778637  10.87282869   2.48020747   7.2903068
   6.75167588  35.18637765]
reading folder84
------------------------------------------
Right a

In [5]:
#JS score with 2-means

def js_2mean(a,b):
    def two_means(x):
        y =  np.asarray(x).reshape(-1,1)
        km = KMeans(2)
        km.fit(y)
        km.cluster_centers_
        return float( (km.cluster_centers_[1]+km.cluster_centers_[0])/2 )

    thershold=0.5
    k_mean_candidates=[]
    for i in range(0,len(a[0])):
        temp=[]
        for c in range(0,len(a)):
            temp.append(a[c][i])        
        k_mean_candidates.append( temp )

    k_means_answer=[]
    for i in k_mean_candidates:
        k_means_answer.append(two_means(np.asarray(i)))

    print("Right answer:",np.asarray(k_means_answer)/ np.asarray(k_means_answer).sum()*1024)
    wrong_out=[]
    result=[]
    for i in range(0, len(a)):
        k = JS(np.asarray(a[i])/np.asarray(a[i]).sum(), np.asarray(k_means_answer)/ np.asarray(k_means_answer).sum())
        result.append([b[i],k])
        if k > thershold:
            print(k)
            print(b[i]+":"+str(a[i])+":"+str(k))
            wrong_out.append([b[i],k])
    return result,wrong_out


js_2means=[]
for i in range(1,100):
    print("reading folder"+str(i))
    print("------------------------------------------")
    a,b=read_d(str(i),noise_file)
    js_2means.append(["folder"+str(i),js_2mean(a,b)])
for i in range(0,len(js_2means)):
    if len(js_2means[i][1][1])>0:
        print(js_2means[i][0])

reading folder1
------------------------------------------
Right answer: [68.10283973 77.08330855 75.93302071 45.77856693 72.79350919 36.05294575
 34.64183668 72.96513878 12.05356085 43.61662462 49.42452375 38.28430524
 44.79441004 33.32955384 36.33189791 50.13059294 46.83789289 26.96144733
 22.64019274 13.73730999 20.21193297 19.68250669 19.3336061  31.65101218
  2.06320023  4.19606487  4.73016569  4.10677231  4.31291474  3.30804631
  3.55354704  5.35675242]
reading folder2
------------------------------------------
Right answer: [155.61628122  78.98847606  55.52797     44.0036598   53.47610514
  46.7151938   50.20978653  57.26266305  55.69462174  26.11897737
  35.85387554  53.90326241  34.39802083  52.36512434  49.75170818
  13.67152388  24.75870462  18.58325375  27.71756562  35.48959646
   5.08094006   4.61568929   5.8283537    6.47828624   5.51888531
   2.77856239   3.52452561   5.19139528   3.48245479   5.77523
   4.60230078   1.0170062 ]
reading folder3
--------------------------

Right answer: [ 11.70464797  17.50398153  12.01395854  25.03381587  24.98654011
  77.02624658  21.0744966   80.47374076  16.31793132  31.55116958
  35.75801125 115.71486166  25.71801566 106.25901615 115.96173466
  92.62413164   4.56029294  15.04616131   3.43502585  15.66957798
  11.72958463  50.95184438   9.61931921  48.81664229   1.46383029
   3.00476444   3.60206812  11.25118427   2.64484522  10.90965318
  11.59200513   9.98090088]
reading folder20
------------------------------------------
Right answer: [63.61947655 77.10467951 71.85967464 38.9853494  68.69400211 34.76314731
 32.89436526 54.50701853 74.24659307 44.8527699  45.50358019 43.39524382
 36.71261901 33.09788474 50.37156336 47.34938979 38.26271184 26.6775953
  8.04541559 20.71953536  6.92847258 12.21529596 24.31601385 30.46821997
  6.04081077  4.2695489   5.37519702  3.96988025  3.82831217  4.15529415
  5.62069116  5.14964796]
reading folder21
------------------------------------------
Right answer: [26.88964104 62.91855815

Right answer: [ 10.7974609   14.54574943  25.8974884   65.0254578   19.91585164
  54.39735682  63.09579247  67.17845128  13.87471319  26.91697453
  27.19123883 101.1466471   18.67291611  93.94072213  91.45210753
  70.15637784   5.08406526  10.87063139  14.91735464  49.3832028
   9.68723555  38.7978638   41.46923681  44.9497864    1.53080364
   2.71104589   2.76402396  10.24868224   1.85768684   9.27299127
   8.67180319   7.57828033]
reading folder38
------------------------------------------
Right answer: [  8.16991861  14.53885304  25.6314293   52.69716547  12.46376313
  20.84106018  51.51832135 119.83033961   9.54673     27.60925544
  28.51089043 105.0146284   23.50658901  93.82814056 103.20601049
  75.89416523   5.95789122   6.17746131  19.80054288  58.40490383
   6.7825888    8.37022532  53.98007973  44.90353247   1.17394236
   2.8778962    2.93535129  10.55016881   2.27390647   9.23669314
   9.74424964   8.02330626]
reading folder39
------------------------------------------
Right

Right answer: [12.14021185 40.77533374 20.30588334 59.07345314 14.15761785 61.90169294
 57.85962852 68.14263362 15.92937084 65.16347824 63.79849069 77.41205922
 55.27953348 72.6599018  76.05416891 44.00800587  8.6337464  33.68437019
  8.56655493 38.62044137  7.25333102 25.6433145  24.7278903  28.34367293
  1.48819207  6.58643186  5.94246596  7.33695292  5.46922007  7.95141941
  4.35619011  4.7343419 ]
reading folder56
------------------------------------------
Right answer: [  6.41198971  18.01320753  13.48657731  45.58782234  13.40334815
  29.26136578  27.0705431  121.89003635   8.80206937  28.46984876
  18.1814518   46.03432251  17.77660879 113.60902697  32.65971105
 153.97118205   3.54480969   9.03444437   7.11792348  27.14368806
   6.57421295  16.60424456  18.76703579  84.97123892   7.74002926
  23.50189274   2.66205037   5.28755086  18.3781663   78.2681238
   3.25466183  16.52081546]
reading folder57
------------------------------------------
Right answer: [37.73858399 56.97697645

Right answer: [  7.32996701  21.53324914  17.7561766   53.93701259  14.92096064
  35.18129504  37.36924987 162.13572942  10.48910292  14.09331526
  17.78164261  57.89485523  21.12830097  32.534952    41.15944126
 203.60076282   4.46079637  15.50474966   9.94235537  30.24725529
   9.35239276  21.81588328  25.95411017 113.37892802   1.14224857
   2.08071463   3.43525886   5.96753539   2.23993161   3.34453619
   4.38439833  21.90289215]
reading folder74
------------------------------------------
Right answer: [89.46672719 50.55399985 59.26224865 70.82328646 56.63635564 49.34724772
 41.55397054 38.61110672 44.30073994 67.02908662 69.23505934 31.92778048
 64.23838007 36.67274428 35.42513441  4.66616897 30.42792839 24.00946822
 37.16809499 22.94183624 34.9098158   5.29055406 22.88932974  3.38242927
  4.65502286  5.50209688  6.80309105  2.62932506  5.97771593  3.58195599
  3.36954402  0.71175466]
reading folder75
------------------------------------------
Right answer: [28.07069631 84.7556861

Right answer: [25.66811247 58.26093415 86.39936481 91.09270054 16.6790796  82.87523842
 85.04048614 24.60638506 46.28539508 82.83601686 83.62034656 29.30773058
 88.67406431 39.43461533 21.44589258  4.33774764 11.6669383   7.27159004
 47.11472277 10.70740429  8.56966584 10.14901195 20.80037853  3.7899246
  4.37479454  7.97959123  7.82315889  2.78950314  8.57092229  3.78080198
  1.50315219  0.54432927]
reading folder92
------------------------------------------
Right answer: [81.76323098 98.30789572 96.91410759 43.9554449  94.07272939 56.95376487
 62.86142502 57.67329645 13.76640641 18.04915215 47.16632793  7.53503273
 43.38416662 51.97939419 49.69596342  5.52280351 51.42671229 14.93675808
  8.73085751  4.31150897 24.09673394 30.03909099 30.50693406  5.19261957
  1.47693324  1.97465434  5.050492    0.78413349  4.42855743  5.24230152
  5.27814285  0.92242783]
reading folder93
------------------------------------------
Right answer: [  8.4580372   18.58584191  28.52962258  44.13560443  14.

In [6]:
####KS score with standardised answers
def formal_ks(a,b):
    thershold=0.6
    candidates = [] # save right answer candidates
    answer = -1
    for results in a:
        flag = -1
        for i in range(0,len(candidates)):
            if ks_score(candidates[i]/(candidates[i].sum()),np.asarray(results)/(np.asarray(results).sum()))<thershold:
                flag = i
                candidates[i] = candidates[i] + np.asarray(results)/(np.asarray(results).sum())
                if candidates[i].sum() > candidates[answer].sum():
                    answer = i
                break

        if flag == -1:
            candidates.append(np.asarray(results)/(np.asarray(results).sum()))
            if answer == -1:
                answer = 0

    wrong_out = []           
    print("Right answer:",candidates[answer]/candidates[answer].sum()*1024)

    result=[]

    for i in range(0, len(a)):
        k = ks_score(np.asarray(a[i])/np.asarray(a[i]).sum(),candidates[answer]/candidates[answer].sum())
        result.append([b[i],k])
        if k > thershold:
            print(k)
            print(b[i]+":"+str(a[i])+":"+str(k))
            wrong_out.append([b[i],k])
            
    return result,wrong_out

ks_error=[]
for i in range(1,100):
    print("reading folder"+str(i))
    print("------------------------------------------")
    a,b=read_d(str(i),noise_file)
    ks_error.append(["folder"+str(i),formal_ks(a,b)])
for i in range(0,len(ks_error)):
    if len(ks_error[i][1][1])>0:
        print(ks_error[i][0])

reading folder1
------------------------------------------
Right answer: [ 47.35204969  81.51453416  49.43503106  58.2757764   76.45813665
  57.76695652  48.60819876  95.53093168  24.58236025  45.65863354
  33.8047205   47.33614907  43.85391304  38.45565217  46.58086957
 113.59403727   9.52447205  10.75677019   7.23478261   7.89465839
  10.48645963   8.74534161   7.87080745  14.17540373   2.4884472
   4.39652174   3.21192547   4.93714286   4.34881988   3.66509317
   4.54757764  10.90782609]
reading folder2
------------------------------------------
Right answer: [189.62285714  87.30236025  48.13913043  60.19975155  56.47900621
  52.60720497  21.84745342  21.48173913  97.61391304  52.75031056
  45.88124224  64.39751553  39.31428571  52.9331677   15.99602484
  17.72919255  19.07279503   9.07925466   7.06782609   8.66583851
   5.96273292   5.54931677   2.71900621   2.83031056   9.65167702
   5.54931677   4.49987578   6.20124224   3.96720497   5.82757764
   1.41515528   1.64571429]
reading

Right answer: [ 47.22484472  32.65987578  61.01863354  43.55975155 104.15701863
  53.42608696  62.95055901  79.35204969  31.6978882   24.9242236
  45.14186335  45.05440994  57.94981366  41.00770186  70.81341615
 116.27329193   8.42732919   5.70037267   7.19503106   6.51130435
  11.25763975   6.13763975   8.14906832  10.49440994   2.90186335
   2.40099379   4.61913043   4.52372671   5.69242236   4.00695652
   7.24273292  11.52795031]
reading folder21
------------------------------------------
Right answer: [ 31.97614907  50.64347826  38.6226087   51.4226087   25.68745342
  33.97962733  26.00546584  37.35850932  92.48596273  82.57192547
  58.2757764   77.57913043  81.88819876  53.26708075  51.34310559
 131.82409938   3.64124224   5.77987578   5.7242236    6.47950311
   2.7826087    3.92745342   4.22956522   6.71801242   9.02360248
   7.69590062   5.45391304   7.10757764   8.16496894   4.90534161
   4.82583851  12.60919255]
reading folder22
------------------------------------------
Right

Right answer: [ 27.7068323   46.78757764  38.12173913 105.4131677   41.65962733
  70.08198758  60.44621118  73.21440994  28.67677019  43.00322981
  40.74534161  94.31453416  62.7915528   63.28447205  76.52968944
  46.50136646   4.39652174   6.0978882    4.30111801  10.56596273
   9.29391304  10.57391304   6.82136646   8.42732919   3.07677019
   4.30906832   3.70484472   9.17465839   6.2489441    5.93888199
   7.17913043   4.61118012]
reading folder41
------------------------------------------
Right answer: [ 29.01863354  33.68546584  58.69714286  52.88546584  66.82236025
  69.63677019 140.9510559   98.82236025  18.95354037  15.69391304
  33.86832298  27.26956522  52.58335404  34.7110559   66.3373913
  66.88596273   7.94236025  10.1684472   14.35031056  12.65689441
  16.47304348  15.39180124  26.37118012  21.51354037   2.07503106
   1.74111801   3.6173913    2.75080745   5.17565217   3.56968944
   6.58285714   6.79751553]
reading folder42
------------------------------------------
Right

Right answer: [ 39.16322981  52.92521739  51.51801242  68.13416149  24.55850932
  36.77018634  24.43130435  18.55602484  98.70310559 132.25341615
 148.42434783  54.94459627  78.27875776  37.06434783  49.57018634
  16.74335404   4.15801242   4.87354037   5.09614907   6.44770186
   2.75875776   3.88770186   2.52819876   1.8842236    9.4847205
  13.06236025  14.23900621   5.15975155   8.02186335   3.47428571
   5.07229814   1.81267081]
reading folder61
------------------------------------------
Right answer: [37.13590062 64.62807453 92.85962733 69.66857143 57.75900621 50.38111801
 66.21018634 82.57192547 26.18832298 32.93813665 48.39354037 76.01291925
 33.86037267 45.88124224 64.4531677  68.60322981  4.86559006  8.82484472
 10.00944099  9.23826087  7.3063354   8.15701863  7.91055901 11.30534161
  2.85416149  3.13242236  4.38062112  7.78335404  3.48223602  4.77018634
  6.14559006  6.28869565]
reading folder62
------------------------------------------
Right answer: [44.10832298 64.69167702

Right answer: [ 36.57142857  42.28987013  40.94337662  16.37402597  22.90701299
  12.61714286  16.10805195   5.45246753 160.16623377 144.40727273
 147.10025974  50.35220779 120.40311688  40.31168831  61.32363636
  16.15792208   3.92311688   3.49090909   4.18909091   1.66233766
   2.65974026   1.31324675   1.51272727   0.58181818  15.36
  13.49818182  14.67844156   4.8374026   11.70285714   3.60727273
   5.81818182   1.67896104]
reading folder82
------------------------------------------
Right answer: [ 46.48        36.53714286  54.59428571  58.99428571  51.58857143
  17.82857143  26.54857143  21.71428571 115.45142857 107.18857143
 125.50857143  50.21714286  89.57714286  36.25142857  32.01142857
  13.06285714  16.30857143   5.29142857   8.77714286   7.37142857
  29.28         5.10857143   9.30285714   3.10857143  11.56571429
  10.14857143  11.55428571   4.70857143  10.10285714   3.45142857
   2.98285714   1.38285714]
reading folder83
------------------------------------------
Right answ

In [7]:
##KS score with trimmed mean
def ks_ternn(a,b):
    thershold=0.6
    candidates = [] # save right answer candidates

    for i in range(0,len(a[0])):
        temp=[]
        for c in range(0,len(a)):
            temp.append(a[c][i])        
        fla=sorted(temp)[4:-4]
        candidates.append( sum(fla)/len(fla)  )
    print("Right answer:", np.asarray(candidates)/ np.asarray(candidates).sum()*1024)
    max_diff_name = ''
    result=[]
    wrong_out = [] 
    for i in range(0, len(a)):
        k = ks_score(np.asarray(a[i])/np.asarray(a[i]).sum(), np.asarray(candidates)/ np.asarray(candidates).sum())
        result.append([b[i],k])
        if k > thershold:
            print(k)
            print(b[i]+":"+str(a[i])+":"+str(k))
            wrong_out.append([b[i],k])
    return result,wrong_out


ks_ter=[]
for i in range(1,100):
    print("reading folder"+str(i))
    print("------------------------------------------")
    a,b=read_d(str(i),noise_file)
    ks_ter.append(["folder"+str(i),ks_ternn(a,b)])
for i in range(0,len(ks_ter)):
    if len(ks_ter[i][1][1])>0:
        print(ks_ter[i][0])

reading folder1
------------------------------------------
Right answer: [ 48.29799992  80.62071203  29.79750023  62.00059704  73.736187
  63.96760419  50.19855413 104.79629317  28.17604839  44.04501149
  23.59079523  52.67060366  41.25398783  41.44005607  51.24851065
 133.9957169    4.69157787   9.34328397   4.30615079   6.83136267
   8.28003686   6.39277324   5.02384259   9.68883928   2.57837424
   4.25298844   2.13978481   5.43585084   3.96059548   3.76123665
   4.90422729  12.57289706]
reading folder2
------------------------------------------
Right answer: [198.05392427  90.51113001  44.09024061  61.04507028  54.78305237
  50.3906758   14.07353309  17.36461408 115.62323045  60.39197639
  44.1286579   63.68305738  39.68505828  47.52218498  12.48561853
  19.36231304  18.56835576   8.27252264   4.49482267   5.58331249
   5.92906808   5.49367215   1.28057626   1.86964134  10.91050973
   6.24921215   4.31554199   6.00590266   3.95698064   5.10949927
   1.01165524   1.75438948]
reading 

Right answer: [ 48.68226476  25.95523144  47.19132989  42.21290388 118.78213309
  54.67193356  68.18109997  88.85971842  27.71842398  19.58958776
  38.80320065  43.0815355   62.73594652  41.13683784  77.54157804
 126.28866606   4.8487795    2.63182417   5.12103717   4.64134508
  11.75893852   5.60072926   6.27489112   8.62149296   2.63182417
   1.90580371   3.6819609    4.48576927   6.00263344   3.83753672
   7.93436645  12.58867619]
reading folder21
------------------------------------------
Right answer: [ 34.70134959  39.43995153  25.54093247  42.50222354  27.86403537
  21.7790897   16.07692803  33.72459042 105.95197154  89.21507109
  62.86897227  86.83917039  87.80273012  58.64514882  53.98574356
 146.88346073   3.44505601   4.97619201   2.62669021   4.07862953
   2.81148249   2.41549904   1.66313049   3.31306152   9.96558347
   8.24965519   5.92655229   7.65568002   8.71163588   5.21378208
   5.21378208  13.91221851]
reading folder22
------------------------------------------
Righ

Right answer: [ 10.68719361  21.4750741   18.65584133  46.14336082  16.35442682
  34.65067214  27.40121645  55.62231181  24.68267056  58.01002936
  42.90699667 108.95759295  45.49608799 112.09327022  58.91621132
 250.2500597    1.2370103    2.58909132   1.98497001   4.81858662
   1.49591943   3.63911169   2.76169741   6.17066764   2.05688921
   5.73915242   4.12816227   9.96800157   4.18569763  11.19062803
   6.18505148  23.54634715]
reading folder39
------------------------------------------
Right answer: [ 88.19006479 119.07795013 112.00314157  62.29098763 107.25309248
  40.3125859   33.28804241  14.56430395 103.01826036  70.99941096
  51.87355193  30.56116238  54.22344394  24.32829374  13.49617122
   9.82682113   9.2990379   11.5735323   10.4551345    6.04437463
   9.73885726   3.84527783   3.14156686   1.34459061   9.47496564
   6.81091694   4.78774789   3.14156686   4.68721775   2.18653053
   1.19379541   0.96760259]
reading folder40
------------------------------------------
Righ

Right answer: [ 16.11078004  25.94233565  35.89181097  54.75896417  35.24325258
  43.45341222  69.83794677 130.44867643  28.07963035  31.8088411
  41.22767774  64.0598811   36.14239035  63.11652344  81.92471679
 168.97894085   1.42977645   2.65319342   4.43672899   6.01390508
   3.99453009   5.29164687   8.81449814  13.60498625   2.55001367
   3.55233119   4.00927006   6.67720344   3.58181112   5.82228556
   8.56391876  15.97812037]
reading folder56
------------------------------------------
Right answer: [  4.8455241   10.71246974  11.89582396  32.49864367  10.2889535
  20.14193439  21.49967764  66.57924508  12.87987641  29.57139903
  30.79212233 115.18396244  27.90224677  67.52592845  81.50196455
 392.43762697   0.52316713   1.19581058   1.13352877   3.40058632
   1.19581058   1.64423954   2.32933935   6.48976365   0.92177065
   2.55355383   2.97707008  10.32632258   2.50372839   5.92922744
   7.05029985  37.56838225]
reading folder57
------------------------------------------
Right 

Right answer: [3.56069640e+00 8.83444263e+00 1.05964367e+01 3.59006297e+01
 5.64082832e+00 1.53195598e+01 1.89903808e+01 6.60503065e+01
 1.09879910e+01 2.63809672e+01 3.47626752e+01 1.15728751e+02
 2.46189731e+01 6.32360104e+01 8.19694337e+01 4.11425622e+02
 3.79318174e-01 1.10124631e+00 1.02782989e+00 3.73200139e+00
 6.36275646e-01 1.55398091e+00 1.98224336e+00 6.42393681e+00
 1.11348238e+00 2.82653220e+00 3.16914216e+00 1.05842007e+01
 2.48392223e+00 6.02014650e+00 8.06357021e+00 3.88984669e+01]
0.60568802928941
startQiskit_noise_224.csv:[70.0, 193.0, 111.0, 409.0, 133.0, 339.0, 286.0, 1443.0, 5.0, 4.0, 3.0, 15.0, 2.0, 12.0, 8.0, 38.0, 54.0, 166.0, 86.0, 317.0, 113.0, 266.0, 257.0, 1215.0, 0.0, 4.0, 2.0, 8.0, 1.0, 5.0, 4.0, 31.0]:0.60568802928941
0.6117594578608385
startQiskit_noise_231.csv:[56.0, 89.0, 176.0, 435.0, 140.0, 289.0, 355.0, 1492.0, 2.0, 3.0, 4.0, 14.0, 4.0, 13.0, 13.0, 50.0, 42.0, 57.0, 127.0, 324.0, 90.0, 206.0, 305.0, 1261.0, 3.0, 1.0, 4.0, 6.0, 2.0, 3.0, 4.0, 30.0]:0

Right answer: [ 32.80433711  60.5444503   57.91662276  44.58605394  58.1602624
  84.50814908  45.36918135  41.05327918  39.97430363  68.27130742
  45.61282099  78.33014395  47.09206166 128.20665862  39.52183002
  51.1643242    3.28913513   6.35203345   6.61307592   4.69876447
   6.1606023   11.24222906   7.84867694   4.40291633   3.7764144
   7.22217501   5.29046073   8.05751092   4.62915314  12.98251219
   4.21148519   4.1070682 ]
reading folder91
------------------------------------------
Right answer: [ 34.54687791  46.78961523  35.44667821  17.16437225  28.60274264
  17.62790574  12.88350419   6.7485022  128.65780855 227.4313407
 113.82473705  40.90001331 119.16900546  58.89601917  33.1699108
  11.77920383   3.85823459   4.89436826   4.0082013    2.98570097
   3.55830116   2.23586739   1.17246705   0.62713354  11.4792704
  21.45887365  10.63400346   3.65373452  11.07027027   5.20793503
   2.48126747   1.03613367]
reading folder92
------------------------------------------
Right ans

In [8]:
######IQR method

def box_line(a,b):
    def box_line(x):
        a=sorted(x)
        if (len(a)+1)%4==0:

            q1 = a[ int((len(a)+1)*0.25)-1]
            q3 = a[ int((len(a)+1)*0.75)-1]
        else:
            q1 = (a[ int((len(a)+1)*0.25)-1] + a[ math.ceil((len(a)+1)*0.25)-1])/2
            q3 = (a[ int((len(a)+1)*0.75)-1] + a[ math.ceil((len(a)+1)*0.75)-1])/2
            print(q3,q1)
        iqr=q3-q1
        return ((q1-1.5*iqr),(q3+1.5*iqr))



    box_candidates=[]
    for i in range(0,len(a[0])):
        temp=[]
        for c in range(0,len(a)):
            temp.append(a[c][i])        
        box_candidates.append( temp )

    box_answer=[]
    for i in box_candidates:
        box_answer.append(box_line(i))

    print("Right answer:",box_answer)
    wrong_out=[]
    result=[]
    for i in range(0,len(a)):
        flag=0
        for c in range(0,len(a[0])):
            if a[i][c]<box_answer[c][0] or  a[i][c]>box_answer[c][1]:
                flag += 1
        result.append([b[i],flag])
        if flag >= 10:
            print(b[i]+":"+str(a[i])+":"+str(flag))
            wrong_out.append([b[i],flag])
    return result,wrong_out


box_l=[]
for i in range(1,100):
    print("reading folder"+str(i))
    print("------------------------------------------")
    a,b=read_d(str(i),noise_file)
    box_l.append(["folder"+str(i),box_line(a,b)])
for i in range(0,len(ks_ter)):
    if len(box_l[i][1][1])>0:
        print(box_l[i][0])

reading folder1
------------------------------------------
Right answer: [(65.0, 417.0), (-1068.5, 2119.5), (-90.0, 446.0), (-133.5, 742.5), (-997.0, 1955.0), (72.5, 548.5), (-220.5, 687.5), (342.5, 698.5), (33.5, 253.5), (-589.0, 1187.0), (23.0, 207.0), (-7.0, 553.0), (-524.5, 1103.5), (-114.0, 526.0), (-227.5, 752.5), (516.5, 824.5), (8.0, 40.0), (-103.5, 204.5), (-84.5, 167.5), (-14.0, 82.0), (-99.0, 197.0), (-1.0, 63.0), (-24.5, 75.5), (15.0, 87.0), (-0.5, 27.5), (-61.5, 118.5), (1.5, 21.5), (-2.5, 57.5), (-42.5, 97.5), (-9.5, 50.5), (-20.0, 68.0), (29.0, 93.0)]
startQiskit_noise_215.csv:[695.0, 394.0, 318.0, 264.0, 279.0, 209.0, 312.0, 510.0, 25.0, 15.0, 10.0, 12.0, 10.0, 6.0, 16.0, 6.0, 616.0, 323.0, 282.0, 199.0, 217.0, 171.0, 222.0, 433.0, 18.0, 5.0, 6.0, 6.0, 8.0, 4.0, 3.0, 6.0]:13
startQiskit_noise_231.csv:[342.0, 190.0, 245.0, 184.0, 1056.0, 592.0, 729.0, 576.0, 15.0, 8.0, 6.0, 4.0, 26.0, 11.0, 19.0, 10.0, 90.0, 88.0, 73.0, 113.0, 257.0, 310.0, 260.0, 339.0, 8.0, 3.0, 1.0, 6

Right answer: [(-13.0, 163.0), (-283.0, 589.0), (-273.0, 599.0), (6.5, 290.5), (-263.5, 524.5), (-119.5, 356.5), (-137.0, 367.0), (-60.5, 599.5), (-271.5, 644.5), (-1582.0, 2938.0), (-1444.5, 2743.5), (14.0, 790.0), (61.0, 269.0), (-98.0, 686.0), (-49.5, 722.5), (880.5, 1308.5), (4.0, 12.0), (-31.5, 60.5), (-40.0, 80.0), (-0.5, 27.5), (-30.0, 58.0), (-14.0, 42.0), (-13.5, 38.5), (-9.5, 58.5), (-29.5, 70.5), (-145.0, 263.0), (-122.0, 230.0), (4.5, 72.5), (-6.5, 45.5), (-22.0, 82.0), (-18.5, 81.5), (72.0, 128.0)]
startQiskit_noise_227.csv:[123.0, 72.0, 493.0, 310.0, 300.0, 246.0, 1478.0, 972.0, 2.0, 3.0, 13.0, 9.0, 6.0, 9.0, 23.0, 34.0, 37.0, 33.0, 122.0, 187.0, 103.0, 94.0, 285.0, 608.0, 0.0, 2.0, 2.0, 7.0, 2.0, 6.0, 3.0, 16.0]:14
reading folder12
------------------------------------------
Right answer: [(6.0, 86.0), (-130.0, 334.0), (-79.5, 268.5), (-178.0, 590.0), (-39.5, 180.5), (-204.5, 527.5), (-191.0, 481.0), (97.5, 437.5), (24.0, 200.0), (-338.5, 865.5), (-256.5, 715.5), (194.5, 

Right answer: [(95.5, 267.5), (-747.5, 1496.5), (-36.0, 276.0), (-406.0, 962.0), (46.0, 230.0), (-305.0, 687.0), (17.0, 145.0), (-61.0, 435.0), (-800.0, 1696.0), (-157.5, 1174.5), (-282.5, 825.5), (-29.0, 891.0), (-919.0, 1809.0), (77.0, 533.0), (-161.5, 746.5), (446.5, 1042.5), (7.5, 27.5), (-78.5, 157.5), (-7.5, 36.5), (-36.0, 84.0), (6.0, 22.0), (-33.5, 66.5), (-7.0, 25.0), (-8.5, 43.5), (-72.0, 160.0), (-29.0, 115.0), (-29.0, 83.0), (-5.0, 83.0), (-88.5, 179.5), (-10.5, 65.5), (-8.0, 64.0), (40.5, 100.5)]
startQiskit_noise_210.csv:[213.0, 191.0, 972.0, 907.0, 193.0, 134.0, 597.0, 657.0, 8.0, 5.0, 21.0, 18.0, 3.0, 1.0, 9.0, 10.0, 74.0, 76.0, 320.0, 276.0, 47.0, 91.0, 284.0, 449.0, 3.0, 4.0, 14.0, 10.0, 0.0, 3.0, 4.0, 6.0]:13
reading folder22
------------------------------------------
Right answer: [(10.0, 82.0), (1.0, 217.0), (32.0, 184.0), (-271.0, 817.0), (-10.5, 185.5), (110.0, 310.0), (71.0, 399.0), (-974.5, 2397.5), (-55.5, 180.5), (0.0, 232.0), (12.0, 244.0), (302.0, 550.0), (

Right answer: [(9.0, 657.0), (-120.0, 408.0), (-1610.0, 3126.0), (-694.5, 1309.5), (-1628.0, 3156.0), (-354.5, 1081.5), (-395.5, 1352.5), (-1487.5, 2708.5), (-282.5, 529.5), (-140.0, 276.0), (-104.5, 251.5), (-24.5, 155.5), (-302.0, 562.0), (-71.5, 172.5), (-19.0, 141.0), (15.0, 183.0), (-5.5, 70.5), (-9.0, 39.0), (-149.0, 291.0), (-25.5, 90.5), (-165.5, 318.5), (-49.0, 119.0), (-39.5, 124.5), (-142.5, 253.5), (-16.5, 35.5), (-12.5, 23.5), (-17.0, 31.0), (-2.0, 14.0), (-32.5, 59.5), (-3.0, 13.0), (-11.0, 29.0), (-3.0, 21.0)]
reading folder33
------------------------------------------
Right answer: [(71.0, 199.0), (-400.0, 1056.0), (-83.0, 557.0), (-975.5, 1852.5), (27.5, 239.5), (-393.5, 898.5), (-242.5, 657.5), (-512.0, 1016.0), (-466.5, 1209.5), (-933.5, 1830.5), (-890.5, 1817.5), (188.0, 652.0), (-466.0, 1030.0), (-552.5, 1155.5), (-567.5, 1164.5), (137.5, 277.5), (2.5, 22.5), (-25.5, 82.5), (-33.0, 95.0), (-100.0, 188.0), (0.5, 28.5), (-33.0, 79.0), (-45.0, 99.0), (-51.0, 101.0), (

Right answer: [(718.0, 1166.0), (-133.0, 1155.0), (-693.5, 1390.5), (-324.0, 716.0), (-397.0, 1115.0), (-519.5, 1052.5), (-549.5, 1078.5), (-20.0, 164.0), (-89.0, 1023.0), (-360.0, 792.0), (-386.5, 785.5), (-1056.0, 1848.0), (-326.0, 754.0), (-876.5, 1583.5), (-70.5, 213.5), (-48.0, 248.0), (63.5, 115.5), (-26.0, 126.0), (-71.0, 145.0), (-52.5, 103.5), (-51.5, 128.5), (-39.0, 89.0), (-57.0, 111.0), (-1.5, 18.5), (-33.0, 127.0), (-39.0, 81.0), (-41.5, 82.5), (-101.0, 179.0), (-33.0, 79.0), (-81.5, 146.5), (-5.0, 19.0), (-7.0, 25.0)]
reading folder43
------------------------------------------
Right answer: [(-27.5, 192.5), (-237.5, 526.5), (-194.5, 505.5), (-186.5, 649.5), (-178.5, 425.5), (-43.0, 253.0), (-118.0, 378.0), (-417.0, 1135.0), (-233.5, 602.5), (-1240.0, 2368.0), (-1332.0, 2668.0), (-301.0, 1267.0), (-5.0, 315.0), (-236.0, 836.0), (-514.5, 1369.5), (670.0, 1166.0), (-4.5, 23.5), (-25.5, 58.5), (-27.5, 64.5), (-15.0, 57.0), (-19.5, 40.5), (-2.5, 25.5), (-18.0, 46.0), (-29.0, 9

Right answer: [(-14.0, 122.0), (-39.5, 164.5), (-37.5, 302.5), (-275.5, 704.5), (-59.5, 216.5), (-38.0, 178.0), (-12.5, 351.5), (109.0, 405.0), (21.5, 201.5), (-169.0, 463.0), (31.5, 707.5), (-792.0, 2320.0), (-134.0, 474.0), (158.0, 342.0), (-858.0, 2270.0), (1225.5, 1845.5), (-0.5, 11.5), (-10.5, 25.5), (-7.5, 36.5), (-25.0, 63.0), (-13.5, 30.5), (-5.0, 19.0), (-23.5, 60.5), (7.0, 39.0), (-5.0, 27.0), (-16.5, 43.5), (10.0, 58.0), (-66.5, 201.5), (-20.0, 52.0), (12.5, 32.5), (-84.0, 220.0), (98.0, 194.0)]
startQiskit_noise_211.csv:[101.0, 4.0, 115.0, 6.0, 203.0, 19.0, 355.0, 19.0, 176.0, 8.0, 556.0, 33.0, 458.0, 41.0, 1712.0, 115.0, 34.0, 2.0, 47.0, 1.0, 83.0, 8.0, 138.0, 16.0, 63.0, 3.0, 248.0, 20.0, 161.0, 20.0, 781.0, 54.0]:13
startQiskit_noise_212.csv:[100.0, 138.0, 285.0, 798.0, 98.0, 184.0, 264.0, 1080.0, 6.0, 2.0, 10.0, 33.0, 5.0, 5.0, 6.0, 33.0, 109.0, 126.0, 224.0, 649.0, 91.0, 141.0, 219.0, 926.0, 2.0, 6.0, 9.0, 17.0, 2.0, 1.0, 8.0, 23.0]:18
startQiskit_noise_218.csv:[154.0,

Right answer: [(44.0, 444.0), (-909.5, 1774.5), (-111.0, 489.0), (-902.5, 1725.5), (-823.0, 1745.0), (-770.0, 1566.0), (-1068.5, 1983.5), (62.0, 406.0), (29.0, 365.0), (-846.0, 1650.0), (-280.5, 691.5), (-282.0, 710.0), (-886.0, 1714.0), (-15.0, 697.0), (-503.0, 1025.0), (137.5, 397.5), (8.5, 36.5), (-81.5, 162.5), (-21.0, 59.0), (-73.0, 143.0), (-89.5, 178.5), (-67.5, 152.5), (-84.0, 164.0), (-3.0, 53.0), (-4.0, 44.0), (-82.0, 158.0), (-40.0, 88.0), (-34.0, 78.0), (-84.5, 167.5), (-6.0, 66.0), (-46.0, 98.0), (14.5, 42.5)]
reading folder63
------------------------------------------
Right answer: [(4.0, 76.0), (-29.0, 235.0), (16.5, 228.5), (111.0, 591.0), (-18.5, 185.5), (77.0, 309.0), (157.5, 321.5), (939.5, 1143.5), (0.5, 76.5), (16.0, 216.0), (6.5, 250.5), (297.5, 605.5), (-24.0, 200.0), (174.5, 298.5), (200.5, 380.5), (1114.0, 1898.0), (-4.0, 12.0), (-8.5, 27.5), (-5.0, 27.0), (14.5, 58.5), (-8.0, 24.0), (-0.5, 35.5), (1.5, 45.5), (28.0, 156.0), (-7.0, 17.0), (3.0, 19.0), (-8.0, 32

Right answer: [(-7.0, 49.0), (0.0, 96.0), (4.5, 112.5), (56.0, 320.0), (4.0, 60.0), (17.5, 141.5), (67.5, 135.5), (214.5, 506.5), (-9.5, 122.5), (99.5, 191.5), (55.5, 331.5), (571.5, 687.5), (-25.0, 279.0), (291.5, 399.5), (346.0, 538.0), (1806.0, 2670.0), (-3.5, 8.5), (-3.5, 16.5), (-6.0, 18.0), (-8.0, 48.0), (-6.5, 13.5), (-1.5, 18.5), (3.0, 19.0), (-5.0, 75.0), (-6.0, 18.0), (1.5, 29.5), (3.5, 31.5), (34.0, 82.0), (-4.5, 31.5), (8.0, 56.0), (25.5, 61.5), (143.5, 283.5)]
startQiskit_noise_224.csv:[70.0, 193.0, 111.0, 409.0, 133.0, 339.0, 286.0, 1443.0, 5.0, 4.0, 3.0, 15.0, 2.0, 12.0, 8.0, 38.0, 54.0, 166.0, 86.0, 317.0, 113.0, 266.0, 257.0, 1215.0, 0.0, 4.0, 2.0, 8.0, 1.0, 5.0, 4.0, 31.0]:26
startQiskit_noise_231.csv:[56.0, 89.0, 176.0, 435.0, 140.0, 289.0, 355.0, 1492.0, 2.0, 3.0, 4.0, 14.0, 4.0, 13.0, 13.0, 50.0, 42.0, 57.0, 127.0, 324.0, 90.0, 206.0, 305.0, 1261.0, 3.0, 1.0, 4.0, 6.0, 2.0, 3.0, 4.0, 30.0]:26
reading folder74
------------------------------------------
Right answer:

Right answer: [(75.0, 331.0), (-24.5, 427.5), (-175.0, 793.0), (-544.5, 1347.5), (-448.5, 1163.5), (24.5, 940.5), (2.5, 1022.5), (7.5, 1011.5), (-189.5, 358.5), (-238.5, 453.5), (-226.5, 425.5), (-848.5, 1459.5), (-534.5, 957.5), (-888.0, 1528.0), (-880.0, 1544.0), (-236.5, 455.5), (-85.5, 182.5), (-74.0, 158.0), (-77.5, 174.5), (-51.5, 136.5), (-55.5, 132.5), (-2.5, 97.5), (19.0, 99.0), (-115.0, 285.0), (-2.0, 22.0), (-9.5, 34.5), (-22.0, 50.0), (-58.5, 113.5), (-40.5, 91.5), (-65.5, 130.5), (-75.5, 144.5), (-11.0, 37.0)]
reading folder89
------------------------------------------
Right answer: [(1.0, 73.0), (-45.5, 174.5), (15.0, 159.0), (-169.5, 546.5), (-62.5, 165.5), (-157.5, 438.5), (-210.0, 502.0), (93.0, 365.0), (6.5, 218.5), (-335.0, 777.0), (-309.5, 870.5), (-1011.0, 2669.0), (-185.5, 618.5), (97.5, 453.5), (-1399.5, 3140.5), (760.5, 1908.5), (-6.5, 13.5), (-7.5, 20.5), (-5.5, 22.5), (-15.0, 49.0), (-4.5, 15.5), (-17.5, 42.5), (-18.0, 46.0), (-1.0, 47.0), (-7.5, 28.5), (-27.5

In [9]:
####CS score with standardised answers
def cross_entropy(a,b):
    thershold=6
    candidates = [] # save right answer candidates
    answer = 1
    for results in a:
        candidates.append(np.asarray(results))
    wrong_out = []      
    result=[] 
    print("Right answer:",candidates[answer]/candidates[answer].sum()*1024)
    for i in range(0, len(a)):
        k = entropy_score(candidates[answer]/candidates[answer].sum(),np.asarray(a[i])/np.asarray(a[i]).sum())-\
        entropy_score(candidates[answer]/candidates[answer].sum(),candidates[answer]/candidates[answer].sum())
        result.append([b[i],k])
        if k > thershold:
            print(k)
            print(b[i]+":"+str(a[i])+":"+str(k))
            wrong_out.append([b[i],k])
    return result,wrong_out


cross_error=[]
for i in range(1,100):
    print("reading folder"+str(i))
    print("------------------------------------------")
    a,b=read_d(str(i),noise_file)
    cross_error.append(["folder"+str(i),cross_entropy(a,b)])
for i in range(0,len(ks_error)):
    if len(cross_error[i][1][1])>0:
        print(cross_error[i][0])

reading folder1
------------------------------------------
Right answer: [ 52.11428571  25.05142857  24.86857143  35.65714286 150.85714286
  61.25714286  56.13714286  90.33142857  32.18285714  15.36
  24.32        38.94857143 102.03428571  30.53714286  70.76571429
 124.16         6.76571429   2.01142857   2.92571429   2.56
  15.54285714   5.48571429   5.30285714   8.41142857   2.01142857
   2.19428571   1.82857143   4.75428571   9.69142857   5.66857143
   5.48571429   8.77714286]
reading folder2
------------------------------------------
Right answer: [184.50285714  84.11428571  43.15428571 112.45714286  30.90285714
  17.18857143  10.42285714  19.74857143 116.48        53.02857143
  76.25142857 115.38285714  24.50285714  12.43428571  13.71428571
  23.04        14.44571429   6.94857143   4.38857143   9.50857143
   4.57142857   1.64571429   1.09714286   1.82857143  12.06857143
   7.31428571   6.58285714  12.25142857   1.64571429   0.36571429
   1.46285714   0.54857143]
reading folder3
--

Right answer: [111.72571429 174.81142857  14.08        12.8        121.78285714
  53.02857143   8.59428571   7.13142857  66.74285714 112.27428571
   8.77714286  11.88571429 125.98857143  81.37142857  15.54285714
   9.50857143   8.22857143  17.92         1.64571429   2.19428571
  11.70285714   5.85142857   0.73142857   0.18285714   7.49714286
   9.69142857   0.18285714   1.82857143  11.33714286   6.76571429
   1.64571429   0.54857143]
reading folder19
------------------------------------------
Right answer: [  7.31428571  18.83428571   6.58285714  17.92        11.52
  44.61714286  15.90857143  45.53142857  21.21142857  85.21142857
  19.56571429  78.62857143  50.10285714 202.60571429  52.48
 243.93142857   0.91428571   3.10857143   1.09714286   2.19428571
   1.28         6.94857143   2.37714286   6.94857143   2.74285714
   6.94857143   1.64571429   7.31428571   7.49714286  21.94285714
   3.65714286  25.41714286]
reading folder20
------------------------------------------
Right answer: [ 

Right answer: [2.96228571e+01 4.20571429e+00 3.07200000e+01 3.65714286e+00
 3.01714286e+01 5.30285714e+00 1.79200000e+01 3.10857143e+00
 1.69325714e+02 2.46857143e+01 2.17417143e+02 2.77942857e+01
 1.99131429e+02 2.57828571e+01 1.30194286e+02 1.79200000e+01
 3.10857143e+00 1.82857143e-01 2.74285714e+00 1.82857143e-01
 2.19428571e+00 0.00000000e+00 1.46285714e+00 7.31428571e-01
 1.31657143e+01 1.28000000e+00 2.19428571e+01 2.56000000e+00
 2.32228571e+01 2.37714286e+00 1.07885714e+01 1.09714286e+00]
7.494237703897175
startQiskit_noise_210.csv:[238.0, 789.0, 290.0, 1107.0, 319.0, 1276.0, 191.0, 707.0, 0.0, 10.0, 2.0, 19.0, 5.0, 6.0, 2.0, 13.0, 29.0, 98.0, 40.0, 139.0, 45.0, 153.0, 23.0, 88.0, 0.0, 0.0, 2.0, 3.0, 2.0, 3.0, 1.0, 0.0]:7.494237703897175
6.468839839223008
startQiskit_noise_228.csv:[344.0, 323.0, 1287.0, 1442.0, 76.0, 64.0, 224.0, 233.0, 2.0, 5.0, 12.0, 13.0, 1.0, 2.0, 5.0, 5.0, 156.0, 100.0, 643.0, 439.0, 28.0, 24.0, 92.0, 55.0, 1.0, 2.0, 10.0, 8.0, 0.0, 0.0, 2.0, 2.0]:6.46883

Right answer: [1.84685714e+01 7.31428571e-01 2.10285714e+01 1.09714286e+00
 3.71200000e+01 3.47428571e+00 6.49142857e+01 3.47428571e+00
 3.21828571e+01 1.46285714e+00 1.01668571e+02 6.03428571e+00
 8.37485714e+01 7.49714286e+00 3.13051429e+02 2.10285714e+01
 6.21714286e+00 3.65714286e-01 8.59428571e+00 1.82857143e-01
 1.51771429e+01 1.46285714e+00 2.52342857e+01 2.92571429e+00
 1.15200000e+01 5.48571429e-01 4.53485714e+01 3.65714286e+00
 2.94400000e+01 3.65714286e+00 1.42811429e+02 9.87428571e+00]
reading folder54
------------------------------------------
Right answer: [ 49.00571429 242.28571429  13.16571429  46.81142857  20.48
  59.97714286  19.38285714  44.61714286  15.90857143  44.25142857
  18.83428571  68.75428571  11.70285714  35.84        40.77714286
 202.05714286   5.30285714  22.49142857   1.82857143   6.58285714
   0.91428571   6.03428571   1.09714286   4.57142857   1.28
   4.93714286   3.29142857   4.75428571   1.28         2.37714286
   4.20571429  19.2       ]
reading fol

Right answer: [ 78.81142857  11.52        97.46285714  14.26285714 110.62857143
  14.81142857  78.62857143  11.52        97.46285714  14.81142857
 155.97714286  20.48       141.71428571  16.27428571  66.01142857
   7.86285714   8.22857143   1.09714286  10.05714286   1.28
  10.97142857   1.28         9.69142857   0.91428571   7.49714286
   0.54857143  12.8          2.19428571  11.88571429   2.19428571
   4.93714286   0.18285714]
reading folder75
------------------------------------------
Right answer: [ 50.46857143 196.38857143  25.96571429 100.93714286  17.18857143
  52.48        12.8         35.10857143  19.56571429  59.61142857
  15.17714286  39.31428571  37.66857143 135.68        26.88
 111.72571429   5.30285714  18.83428571   1.82857143   9.50857143
   2.37714286   3.29142857   1.09714286   3.29142857   2.19428571
   5.66857143   0.91428571   4.20571429   3.47428571  10.42285714
   2.74285714  11.88571429]
reading folder76
------------------------------------------
Right answer: [4

Right answer: [ 57.23428571  43.52        45.89714286  39.31428571 148.84571429
 125.62285714 136.41142857 120.13714286   1.46285714   0.73142857
   0.73142857   1.64571429   3.65714286   2.74285714   2.56
   2.37714286  24.50285714  22.85714286  16.82285714  12.06857143
  72.04571429  70.21714286  40.41142857  27.06285714   0.54857143
   0.36571429   0.18285714   0.18285714   1.28         2.01142857
   0.36571429   0.18285714]
reading folder89
------------------------------------------
Right answer: [  6.21714286  18.65142857  22.30857143  67.65714286  11.52
  45.53142857  56.13714286 213.94285714   3.84        19.2
  14.81142857  71.31428571  14.81142857  50.10285714  56.13714286
 260.02285714   0.54857143   2.01142857   1.46285714   7.31428571
   0.54857143   4.57142857   7.86285714  17.73714286   0.54857143
   1.46285714   1.82857143   6.58285714   2.37714286   4.93714286
   5.48571429  26.51428571]
reading folder90
------------------------------------------
Right answer: [2.194285

In [10]:
##CS score with trimmed mean
def cs_ter(a,b):
    thershold=3
    candidates = [] # save right answer candidates

    for i in range(0,len(a[0])):
        temp=[]
        for c in range(0,len(a)):
            temp.append(a[c][i])        
        fla=sorted(temp)[4:-4]
        candidates.append( sum(fla)/len(fla)  )
    print("Right answer:", np.asarray(candidates)/ np.asarray(candidates).sum()*1024)
    max_diff_name = ''
    result=[]
    wrong_out = [] 
    for i in range(0, len(a)):
        k = entropy_score(np.asarray(a[i])/np.asarray(a[i]).sum(), np.asarray(candidates)/ np.asarray(candidates).sum())-\
        entropy_score(np.asarray(candidates)/ np.asarray(candidates).sum(), np.asarray(candidates)/ np.asarray(candidates).sum())
        k= abs(k)
        result.append([b[i],k])
        if k > thershold:
            print(k)
            print(b[i]+":"+str(a[i])+":"+str(k))
            wrong_out.append([b[i],k])
    return result,wrong_out

cs_ternn=[]
for i in range(1,100):
    print("reading folder"+str(i))
    print("------------------------------------------")
    a,b=read_d(str(i),noise_file)
    cs_ternn.append(["folder"+str(i),cs_ter(a,b)])
for i in range(0,len(ks_error)):
    if len(cs_ternn[i][1][1])>0:
        print(cs_ternn[i][0])

reading folder1
------------------------------------------
Right answer: [ 48.29799992  80.62071203  29.79750023  62.00059704  73.736187
  63.96760419  50.19855413 104.79629317  28.17604839  44.04501149
  23.59079523  52.67060366  41.25398783  41.44005607  51.24851065
 133.9957169    4.69157787   9.34328397   4.30615079   6.83136267
   8.28003686   6.39277324   5.02384259   9.68883928   2.57837424
   4.25298844   2.13978481   5.43585084   3.96059548   3.76123665
   4.90422729  12.57289706]
reading folder2
------------------------------------------
Right answer: [198.05392427  90.51113001  44.09024061  61.04507028  54.78305237
  50.3906758   14.07353309  17.36461408 115.62323045  60.39197639
  44.1286579   63.68305738  39.68505828  47.52218498  12.48561853
  19.36231304  18.56835576   8.27252264   4.49482267   5.58331249
   5.92906808   5.49367215   1.28057626   1.86964134  10.91050973
   6.24921215   4.31554199   6.00590266   3.95698064   5.10949927
   1.01165524   1.75438948]
reading 

Right answer: [  7.02959391  12.87184758  16.91959839  35.40432709   8.09550162
  18.02598361  23.74680475  43.37839618  24.988115    53.22792316
  69.81020898 155.09631855  36.34880228  65.10132553  94.13719135
 267.61029857   0.72859515   1.53814531   1.63259283   3.87234828
   0.71510264   1.83498037   2.45563549   4.76285345   2.29372546
   5.18112104   6.94863889  14.86873798   3.61599072   6.65180383
   9.4852294   25.62226263]
3.0769702663662977
startQiskit_noise_228.csv:[136.0, 290.0, 97.0, 262.0, 319.0, 1021.0, 240.0, 843.0, 4.0, 13.0, 11.0, 17.0, 10.0, 19.0, 1.0, 16.0, 63.0, 212.0, 42.0, 209.0, 167.0, 732.0, 127.0, 685.0, 3.0, 7.0, 1.0, 12.0, 5.0, 17.0, 2.0, 17.0]:3.0769702663662977
reading folder20
------------------------------------------
Right answer: [ 48.68226476  25.95523144  47.19132989  42.21290388 118.78213309
  54.67193356  68.18109997  88.85971842  27.71842398  19.58958776
  38.80320065  43.0815355   62.73594652  41.13683784  77.54157804
 126.28866606   4.8487795 

Right answer: [ 12.86636258  18.27690739  24.49975919  40.68787716  20.48174066
  31.33184123  47.91160722  73.77488172  21.35206958  32.82590588
  53.77182197  73.41224467  45.75029039  52.79995467 132.78318271
 236.77298354   2.07428393   2.828569     3.29274443   6.61449982
   2.81406352   4.56922685   6.49845596  15.12921778   2.08878942
   2.9881293    5.07691872   7.49933422   4.13406238   4.84483101
  12.43119812  21.81624501]
reading folder38
------------------------------------------
Right answer: [ 10.68719361  21.4750741   18.65584133  46.14336082  16.35442682
  34.65067214  27.40121645  55.62231181  24.68267056  58.01002936
  42.90699667 108.95759295  45.49608799 112.09327022  58.91621132
 250.2500597    1.2370103    2.58909132   1.98497001   4.81858662
   1.49591943   3.63911169   2.76169741   6.17066764   2.05688921
   5.73915242   4.12816227   9.96800157   4.18569763  11.19062803
   6.18505148  23.54634715]
reading folder39
------------------------------------------
Righ

Right answer: [ 57.24519903  88.96858015  17.70360785  36.10952948  22.95108442
  28.09255137  19.22749625  48.67167038 130.49785186  42.61587039
  31.68362752  62.50592681  37.23588177  38.32248046  45.3191159
 207.64635851   6.0955536   13.15844505   2.91526477   5.92328795
   3.061028     4.39939955   3.61757855  11.88632952  13.07893783
   4.21388271   3.10078161   5.43299343   3.39230809   3.56457374
   4.42590196  20.9369015 ]
reading folder55
------------------------------------------
Right answer: [ 16.11078004  25.94233565  35.89181097  54.75896417  35.24325258
  43.45341222  69.83794677 130.44867643  28.07963035  31.8088411
  41.22767774  64.0598811   36.14239035  63.11652344  81.92471679
 168.97894085   1.42977645   2.65319342   4.43672899   6.01390508
   3.99453009   5.29164687   8.81449814  13.60498625   2.55001367
   3.55233119   4.00927006   6.67720344   3.58181112   5.82228556
   8.56391876  15.97812037]
reading folder56
------------------------------------------
Right 

Right answer: [3.39206063e+02 9.78204118e+01 7.51139253e+01 1.97963803e+01
 7.71688687e+01 2.35999402e+01 1.82264546e+01 5.23308571e+00
 1.71568629e+02 2.80544448e+01 2.07153612e+01 5.75639428e+00
 3.55339283e+01 7.83686493e+00 5.67981253e+00 1.53163484e+00
 3.27386947e+01 1.01343172e+01 7.74751957e+00 1.83796181e+00
 8.10490103e+00 2.47614299e+00 2.02941616e+00 6.38181184e-01
 1.54950391e+01 2.69312459e+00 1.95283442e+00 5.74363065e-01
 3.30577853e+00 7.78581044e-01 5.61599442e-01 8.93453657e-02]
reading folder72
------------------------------------------
Right answer: [ 33.78608311  32.63732481  72.62355545  76.91959675  58.22473568
  54.98303418 124.91566265 225.55003688  35.21809688  10.3545611
  26.97221539  18.00245881  45.88738628  16.47602656  66.21883452
  32.57437915   3.19449225   3.47774773   7.36464224   7.77378903
   6.08999262   5.52348168  12.38455864  21.70051635   3.43053848
   1.13302188   2.73813622   1.63658716   4.46914187   1.63658716
   6.73518564   3.36759282]


Right answer: [  7.56060767  12.56475088  17.96324232  32.43174333  12.41517051
  20.11176033  29.52172527  46.22033358  22.24668012  41.02581536
  65.84255816 119.13396367  46.94103899  64.02039732 130.89641984
 265.96749177   0.76150005   1.40061617   1.83575906   3.38595559
   1.04706257   2.03973229   2.84202698   4.52820567   2.18931265
   3.78030384   6.3367683   10.83777754   4.106661     6.10559864
  12.56475088  25.37426963]
reading folder90
------------------------------------------
Right answer: [ 32.80433711  60.5444503   57.91662276  44.58605394  58.1602624
  84.50814908  45.36918135  41.05327918  39.97430363  68.27130742
  45.61282099  78.33014395  47.09206166 128.20665862  39.52183002
  51.1643242    3.28913513   6.35203345   6.61307592   4.69876447
   6.1606023   11.24222906   7.84867694   4.40291633   3.7764144
   7.22217501   5.29046073   8.05751092   4.62915314  12.98251219
   4.21148519   4.1070682 ]
reading folder91
------------------------------------------
Right 

In [11]:
#CS score with  2-means

def cs_2mean(a,b):
    def two_means(x):
        y =  np.asarray(x).reshape(-1,1)
        km = KMeans(2)
        km.fit(y)
        km.cluster_centers_
        return float( (km.cluster_centers_[1]+km.cluster_centers_[0])/2 )

    thershold=0.6
    k_mean_candidates=[]
    for i in range(0,len(a[0])):
        temp=[]
        for c in range(0,len(a)):
            temp.append(a[c][i])        
        k_mean_candidates.append( temp )

    k_means_answer=[]
    for i in k_mean_candidates:
        k_means_answer.append(two_means(np.asarray(i)))

    print("Right answer:",np.asarray(k_means_answer)/ np.asarray(k_means_answer).sum()*1024)
    wrong_out=[]
    result=[]
    for i in range(0, len(a)):
        k = entropy_score(np.asarray(a[i])/np.asarray(a[i]).sum(), np.asarray(k_means_answer)/ np.asarray(k_means_answer).sum())-\
        entropy_score(np.asarray(k_means_answer)/ np.asarray(k_means_answer).sum(), np.asarray(k_means_answer)/ np.asarray(k_means_answer).sum())
        k= abs(k)
        result.append([b[i],k])
        if k > thershold:
            print(k)
            print(b[i]+":"+str(a[i])+":"+str(k))
            wrong_out.append([b[i],k])
    return result,wrong_out

cs_2ms=[]
for i in range(1,100):
    print("reading folder"+str(i))
    print("------------------------------------------")
    a,b=read_d(str(i),noise_file)
    cs_2ms.append(["folder"+str(i),cs_2mean(a,b)])
for i in range(0,len(ks_error)):
    if len(cs_2ms[i][1][1])>0:
        print(cs_2ms[i][0])

reading folder1
------------------------------------------
Right answer: [67.96835475 76.93108953 75.78307321 45.68816645 72.64976139 38.0038782
 34.57342827 72.82105205 12.02975828 43.5304934  49.32692347 38.20870393
 44.705953   33.26373685 36.26015208 50.03159836 46.74540052 26.90820565
 22.59548439 13.71018246 20.17201978 19.64363898 19.29542737 31.58850985
  2.05912596  4.18777875  4.72082487  4.09866252  4.30439788  3.3015138
  3.54652973  5.34617426]
reading folder2
------------------------------------------
Right answer: [155.61628122  78.98847606  55.52797     44.0036598   53.47610514
  46.7151938   50.20978653  57.26266305  55.69462174  26.11897737
  35.85387554  53.90326241  34.39802083  52.36512434  49.75170818
  13.67152388  24.75870462  18.58325375  27.71756562  35.48959646
   5.08094006   4.61568929   5.8283537    6.47828624   5.51888531
   2.77856239   3.52452561   5.19139528   3.48245479   5.77523
   4.60230078   1.0170062 ]
reading folder3
----------------------------

Right answer: [ 11.70464797  17.50398153  12.01395854  25.03381587  24.98654011
  77.02624658  21.0744966   80.47374076  16.31793132  31.55116958
  35.75801125 115.71486166  25.71801566 106.25901615 115.96173466
  92.62413164   4.56029294  15.04616131   3.43502585  15.66957798
  11.72958463  50.95184438   9.61931921  48.81664229   1.46383029
   3.00476444   3.60206812  11.25118427   2.64484522  10.90965318
  11.59200513   9.98090088]
reading folder20
------------------------------------------
Right answer: [63.61947655 77.10467951 71.85967464 38.9853494  68.69400211 34.76314731
 32.89436526 54.50701853 74.24659307 44.8527699  45.50358019 43.39524382
 36.71261901 33.09788474 50.37156336 47.34938979 38.26271184 26.6775953
  8.04541559 20.71953536  6.92847258 12.21529596 24.31601385 30.46821997
  6.04081077  4.2695489   5.37519702  3.96988025  3.82831217  4.15529415
  5.62069116  5.14964796]
reading folder21
------------------------------------------
Right answer: [26.8990415  62.94055411

Right answer: [ 10.7974609   14.54574943  25.8974884   65.0254578   19.91585164
  54.39735682  63.09579247  67.17845128  13.87471319  26.91697453
  27.19123883 101.1466471   18.67291611  93.94072213  91.45210753
  70.15637784   5.08406526  10.87063139  14.91735464  49.3832028
   9.68723555  38.7978638   41.46923681  44.9497864    1.53080364
   2.71104589   2.76402396  10.24868224   1.85768684   9.27299127
   8.67180319   7.57828033]
reading folder38
------------------------------------------
Right answer: [  8.16991861  14.53885304  25.6314293   52.69716547  12.46376313
  20.84106018  51.51832135 119.83033961   9.54673     27.60925544
  28.51089043 105.0146284   23.50658901  93.82814056 103.20601049
  75.89416523   5.95789122   6.17746131  19.80054288  58.40490383
   6.7825888    8.37022532  53.98007973  44.90353247   1.17394236
   2.8778962    2.93535129  10.55016881   2.27390647   9.23669314
   9.74424964   8.02330626]
reading folder39
------------------------------------------
Right

Right answer: [12.14021185 40.77533374 20.30588334 59.07345314 14.15761785 61.90169294
 57.85962852 68.14263362 15.92937084 65.16347824 63.79849069 77.41205922
 55.27953348 72.6599018  76.05416891 44.00800587  8.6337464  33.68437019
  8.56655493 38.62044137  7.25333102 25.6433145  24.7278903  28.34367293
  1.48819207  6.58643186  5.94246596  7.33695292  5.46922007  7.95141941
  4.35619011  4.7343419 ]
reading folder56
------------------------------------------
Right answer: [  6.37510152  17.90957752  13.40898901  45.32555552  13.32623867
  29.09302509  26.91480622 121.18880278   8.75143105  33.12343119
  19.15055796  45.76948698  17.67433993 112.95543406  32.47181968
 153.0853856    3.5244164    8.98246919   7.07697405  26.98753038
   6.5363915   16.50872031  18.659069    84.48239925   7.69550086
  23.36668632   2.64673559   5.25713157  18.27243668  77.81784716
   3.23593775  16.42577118]
reading folder57
------------------------------------------
Right answer: [37.73858399 56.9769764

Right answer: [  7.32996701  21.53324914  17.7561766   53.93701259  14.92096064
  35.18129504  37.36924987 162.13572942  10.48910292  14.09331526
  17.78164261  57.89485523  21.12830097  32.534952    41.15944126
 203.60076282   4.46079637  15.50474966   9.94235537  30.24725529
   9.35239276  21.81588328  25.95411017 113.37892802   1.14224857
   2.08071463   3.43525886   5.96753539   2.23993161   3.34453619
   4.38439833  21.90289215]
reading folder74
------------------------------------------
Right answer: [89.46672719 50.55399985 59.26224865 70.82328646 56.63635564 49.34724772
 41.55397054 38.61110672 44.30073994 67.02908662 69.23505934 31.92778048
 64.23838007 36.67274428 35.42513441  4.66616897 30.42792839 24.00946822
 37.16809499 22.94183624 34.9098158   5.29055406 22.88932974  3.38242927
  4.65502286  5.50209688  6.80309105  2.62932506  5.97771593  3.58195599
  3.36954402  0.71175466]
reading folder75
------------------------------------------
Right answer: [28.04514332 84.6785323

Right answer: [25.66811247 58.26093415 86.39936481 91.09270054 16.6790796  82.87523842
 85.04048614 24.60638506 46.28539508 82.83601686 83.62034656 29.30773058
 88.67406431 39.43461533 21.44589258  4.33774764 11.6669383   7.27159004
 47.11472277 10.70740429  8.56966584 10.14901195 20.80037853  3.7899246
  4.37479454  7.97959123  7.82315889  2.78950314  8.57092229  3.78080198
  1.50315219  0.54432927]
reading folder92
------------------------------------------
Right answer: [81.54613788 98.04687416 96.65678674 43.83873689 93.82295281 59.52141125
 62.69451892 57.52016558 13.72985463 18.00122906 47.04109457  7.51502614
 43.26897544 51.84138145 49.56401352  5.50813968 51.290167   14.89709884
  8.70767584  4.3000613  24.03275365 29.9593329  30.42593378  5.17883242
  1.47301178  1.96941135  5.03708222  0.7820515   4.41679897  5.22838245
  5.26412862  0.91997866]
reading folder93
------------------------------------------
Right answer: [  8.36660359  18.38492407  34.15404951  43.65848694  14.